In [4]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

In [24]:
stream= open("Spotify.yaml")
spotify_details = yaml.safe_load(stream)
auth_manager = SpotifyClientCredentials(client_id=spotify_details['Client_id'],
                                        client_secret=spotify_details['client_secret'])
sp = spotipy.client.Spotify(auth_manager=auth_manager)

In [32]:
import json
# Load playlist links from JSON file
with open('playlists.json') as f:
    playlists = json.load(f)['playlists']

# Extract track URIs from each playlist
all_track_uris = []
for playlist_link in playlists:
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    all_track_uris.extend(track_uris)


In [81]:
# Create a dictionary to store track details
track_details = {}

# Extract track details and club them together
for track_uri in all_track_uris:
    track = sp.track(track_uri)
    track_id = track['id']
    track_name = track['name']
    album = track['album']['name']
    track_pop = track['popularity']

    artists_info = []
    for artist in track['artists']:
        artist_uri = artist['uri']
        artist_info = sp.artist(artist_uri)
        artist_name = artist['name']
        artist_pop = artist_info['popularity']
        artist_genres = artist_info['genres']
        artists_info.append({
            "artist_name": artist_name,
            "artist_popularity": artist_pop,
            "artist_genres": artist_genres
        })
        
    audio_features = json.dumps(sp.audio_features(track_uri)[0])

    # Club track details together
    track_details.setdefault(track_name, []).append({
        "track_id": track_id,
        "track_name": track_name,
        "album": album,
        "track_popularity": track_pop,
        "artists_info": artists_info,
        "Audio features": audio_features
    })

In [82]:
len(track_details)

362

In [83]:
track_details

{'Elangathu - Duet': [{'track_id': '01yHYYvFwxQorNJTBwbvPn',
   'track_name': 'Elangathu - Duet',
   'album': 'Pithamagan (Original Motion Picture Soundtrack)',
   'track_popularity': 52,
   'artists_info': [{'artist_name': 'Shreya Ghoshal',
     'artist_popularity': 80,
     'artist_genres': ['filmi', 'modern bollywood']},
    {'artist_name': 'Palani Barathi',
     'artist_popularity': 33,
     'artist_genres': []},
    {'artist_name': 'Sriram Parthasarathy',
     'artist_popularity': 51,
     'artist_genres': ['filmi']}],
   'Audio features': '{"danceability": 0.694, "energy": 0.479, "key": 0, "loudness": -8.918, "mode": 1, "speechiness": 0.0307, "acousticness": 0.358, "instrumentalness": 0, "liveness": 0.194, "valence": 0.739, "tempo": 143.973, "type": "audio_features", "id": "01yHYYvFwxQorNJTBwbvPn", "uri": "spotify:track:01yHYYvFwxQorNJTBwbvPn", "track_href": "https://api.spotify.com/v1/tracks/01yHYYvFwxQorNJTBwbvPn", "analysis_url": "https://api.spotify.com/v1/audio-analysis/01yH

In [84]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({
    "Track_id" : [track["track_id"] for track_details in track_details.values() for track in track_details],
    "Track Name": [track["track_name"] for track_details in track_details.values() for track in track_details],
    "Album": [track["album"] for track_details in track_details.values() for track in track_details],
    "Track Popularity": [track["track_popularity"] for track_details in track_details.values() for track in track_details],
    "Artists": [[artist["artist_name"] for artist in track["artists_info"]] for track_details in track_details.values() for track in track_details],
    "Audio features" : [track["Audio features"] for track_details in track_details.values() for track in track_details]
})

df

,Track_id,Track Name,Album,Track Popularity,Artists,Audio features
0,01yHYYvFwxQorNJTBwbvPn,Elangathu - Duet,Pithamagan (Original Motion Picture Soundtrack),52,"[Shreya Ghoshal, Palani Barathi, Sriram Partha...","{""danceability"": 0.694, ""energy"": 0.479, ""key""..."
1,6NwEpX2JUWHVKWfDDfcqC4,Idhazhin Oram - The Innocence of Love,3 (Original Motion Picture Soundtrack),68,"[Anirudh Ravichander, Ajesh]","{""danceability"": 0.65, ""energy"": 0.783, ""key"":..."
2,43zT3oDEP3I5FiZWugfqCZ,Adada Adada,Santhosh Subramaniyam (Original Motion Picture...,58,"[Siddharth, Devi Sri Prasad]","{""danceability"": 0.876, ""energy"": 0.834, ""key""..."
3,0ew71PqPi7ehntkabL9W4c,Yethi Yethi,Vaaranam Aayiram (Original Motion Picture Soun...,61,"[Harris Jayaraj, Benny Dayal, Naresh Iyer, Sol...","{""danceability"": 0.941, ""energy"": 0.771, ""key""..."
4,5oFnrSf3YWf4AO6rIy9f96,CEO in the House,Sarkar (Tamil) [Original Motion Picture Soundt...,45,"[Nakul Abhyankar, Blaaze]","{""danceability"": 0.659, ""energy"": 0.895, ""key""..."
...,...,...,...,...,...,...
368,6Vh03bkEfXqekWp7Y1UBRb,Live While We're Young,Take Me Home (Expanded Edition),72,[One Direction],"{""danceability"": 0.663, ""energy"": 0.857, ""key""..."
369,4nVBt6MZDDP6tRVdQTgxJg,Story of My Life,Midnight Memories (Deluxe),84,[One Direction],"{""danceability"": 0.6, ""energy"": 0.663, ""key"": ..."
370,6VrLYoQKdhu1Jruei06t65,Somebody To You,Meet The Vamps,71,[The Vamps],"{""danceability"": 0.525, ""energy"": 0.818, ""key""..."
371,6M31fPFCYB8Job3MCjjrDV,They Don't Know About Us,Take Me Home (Expanded Edition),72,[One Direction],"{""danceability"": 0.514, ""energy"": 0.727, ""key""..."


In [85]:

# Combine all track information except the track name and track ID into a single sentence
df['tags'] = df.apply(lambda row: f"Album: {row['Album']}, Track Popularity: {row['Track Popularity']}, Artists: {', '.join(row['Artists'])}, {row['Audio features']}", axis=1)

# Print the DataFrame
df.head()

,Track_id,Track Name,Album,Track Popularity,Artists,Audio features,tags
0,01yHYYvFwxQorNJTBwbvPn,Elangathu - Duet,Pithamagan (Original Motion Picture Soundtrack),52,"[Shreya Ghoshal, Palani Barathi, Sriram Partha...","{""danceability"": 0.694, ""energy"": 0.479, ""key""...",Album: Pithamagan (Original Motion Picture Sou...
1,6NwEpX2JUWHVKWfDDfcqC4,Idhazhin Oram - The Innocence of Love,3 (Original Motion Picture Soundtrack),68,"[Anirudh Ravichander, Ajesh]","{""danceability"": 0.65, ""energy"": 0.783, ""key"":...","Album: 3 (Original Motion Picture Soundtrack),..."
2,43zT3oDEP3I5FiZWugfqCZ,Adada Adada,Santhosh Subramaniyam (Original Motion Picture...,58,"[Siddharth, Devi Sri Prasad]","{""danceability"": 0.876, ""energy"": 0.834, ""key""...",Album: Santhosh Subramaniyam (Original Motion ...
3,0ew71PqPi7ehntkabL9W4c,Yethi Yethi,Vaaranam Aayiram (Original Motion Picture Soun...,61,"[Harris Jayaraj, Benny Dayal, Naresh Iyer, Sol...","{""danceability"": 0.941, ""energy"": 0.771, ""key""...",Album: Vaaranam Aayiram (Original Motion Pictu...
4,5oFnrSf3YWf4AO6rIy9f96,CEO in the House,Sarkar (Tamil) [Original Motion Picture Soundt...,45,"[Nakul Abhyankar, Blaaze]","{""danceability"": 0.659, ""energy"": 0.895, ""key""...",Album: Sarkar (Tamil) [Original Motion Picture...


In [86]:
new = df[['Track_id', 'Track Name', 'tags']]
new.head()

,Track_id,Track Name,tags
0,01yHYYvFwxQorNJTBwbvPn,Elangathu - Duet,Album: Pithamagan (Original Motion Picture Sou...
1,6NwEpX2JUWHVKWfDDfcqC4,Idhazhin Oram - The Innocence of Love,"Album: 3 (Original Motion Picture Soundtrack),..."
2,43zT3oDEP3I5FiZWugfqCZ,Adada Adada,Album: Santhosh Subramaniyam (Original Motion ...
3,0ew71PqPi7ehntkabL9W4c,Yethi Yethi,Album: Vaaranam Aayiram (Original Motion Pictu...
4,5oFnrSf3YWf4AO6rIy9f96,CEO in the House,Album: Sarkar (Tamil) [Original Motion Picture...


In [88]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.63740112, 0.63043478, ..., 0.60708656, 0.61064722,
        0.61427127],
       [0.63740112, 1.        , 0.63740112, ..., 0.61379491, 0.61739491,
        0.621059  ],
       [0.63043478, 0.63740112, 1.        , ..., 0.60708656, 0.61064722,
        0.61427127],
       ...,
       [0.60708656, 0.61379491, 0.60708656, ..., 1.        , 0.63158975,
        0.63533809],
       [0.61064722, 0.61739491, 0.61064722, ..., 0.63158975, 1.        ,
        0.63906444],
       [0.61427127, 0.621059  , 0.61427127, ..., 0.63533809, 0.63906444,
        1.        ]])

In [92]:
from fuzzywuzzy import process

def recommend(song):
    # Use fuzzy matching to find the closest match
    closest_match = process.extractOne(song, new['Track Name'])[0]
    index = new[new['Track Name'] == closest_match].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_songs = []
    for i in distances[1:3]:
        track_id_detail = new.iloc[int(i[0])]['Track_id']
        track_name_detail = new.iloc[int(i[0])]['Track Name']
        recommended_songs.append([track_name_detail, track_id_detail])
    return recommended_songs

In [100]:
recommendations = recommend('kesariya')
print(recommendations)

[['Tum Hi Ho', '56zZ48jdyY2oDXHVnwg5Di'], ['Khairiyat', '5O932cZmzOZGOGZz9RHx20']]


In [101]:
import pickle
pickle.dump(new,open('song_list.pkl','wb'))
pickle.dump(similarity,open('song_similarity.pkl','wb'))

In [ ]:
import streamlit as st

# Define the link you want the button to go to
link = "https://www.example.com"

# Create the button with the desired styling
button_html = f"""
<a href="{link}" target="_blank" style="text-decoration:none;">
  <button class="green-button">
    Open in Spotify
  </button>
</a>
"""

# Display the button using st.markdown
st.markdown(
    """
    <style>
    .green-button {
        background-color: rgb(29, 185, 84);
        font-weight: bold;
        color: black;
        border: black;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 16px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 15px;
    }
    </style>
    """, unsafe_allow_html=True)

st.markdown(button_html, unsafe_allow_html=True)